In [20]:
import pandas as pd

book = pd.read_csv("raw/books_data.csv")
book = book.groupby("categories").filter(lambda x: len(x) >= 10000)
print(book['categories'])

3         ['Fiction']
24        ['Fiction']
49        ['Fiction']
52        ['Fiction']
53        ['Fiction']
             ...     
212352    ['Fiction']
212361    ['Fiction']
212365    ['Fiction']
212394    ['Fiction']
212402    ['Fiction']
Name: categories, Length: 23419, dtype: object


In [21]:
book = pd.read_csv("raw/books_data.csv")
book = book.groupby("categories").filter(lambda x: len(x) >= 1000)
book = book[book['categories'] != 'Fiction']
book = book.dropna(subset=['ratingsCount'])
book = book.groupby("categories").head(20)

print(f"Distinct categories: {book['categories'].nunique()}")
print(f"Number of books left after removing 'Fiction': {len(book)}")
book.to_csv("books_data.csv", index=False)


Distinct categories: 35
Number of books left after removing 'Fiction': 700


In [22]:
import pandas as pd
rating = pd.read_csv("raw/Books_rating.csv")

rating = rating[rating['Title'].isin(book['Title'])]
rating = rating.groupby("Title").head(100)

print(f"Distinct books left: {rating['Title'].nunique()}")
print(f"Number of ratings left: {len(rating)}")

# save the ratings
rating.to_csv("Books_rating.csv", index=False)
print(f"Number of ratings after truncation: {len(rating)}")

Distinct books left: 700
Number of ratings left: 12161
Number of ratings after truncation: 12161


In [ ]:
import openai
openai.api_key = ''

# according to 

prompt = """
Please write a concise English summary (no more than 100 words) capturing the main points and overall positive and negative feedback from the following user reviews:
{allComments}
Begin the summary:
"""

# iterate through each book and generate a summary, it should read all the comments for each book and summarize them
def generate_summary(comments):
    all_comments = "\n".join(comments)
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt.format(allComments=all_comments)}
        ]
    )
    return response['choices'][0]['message']['content']

books = pd.read_csv("books_data.csv")
comments = pd.read_csv("Books_rating.csv")
i = 0
for book_title in books['Title'].unique():
    comments = rating[rating['Title'] == book_title]['review/text'].tolist()
    summary = generate_summary(comments)
    books.loc[books['Title'] == book_title, 'ai_summary'] = summary
    print('summary generated for book:', summary)
    print(f"Processed book {i+1}/{len(books['Title'].unique())}: {book_title}")
    i += 1
books.to_csv("books_data_with_summary.csv", index=False)
print("Summaries generated and saved to books_data_with_summary.csv")

summary generated for book: Everett Ferguson's book on ecclesiology offers a comprehensive exploration of the theological perspective of the Churches of Christ, divided into six chapters that systematically explore various topics related to the church. The book is praised for its insightful exegesis, particularly of Matthew 16:13-23, and its clear focus on central themes like the nature of the church, salvation, worship, ministry, and ethics. Positive feedback highlights the book’s depth, thorough research, rich exegesis, and its utility as a resource for ministers and students. Criticisms point to occasional excessive technical detail and insufficient coverage of certain debated topics like miraculous gifts and women's roles. Overall, it's recommended for serious students of ecclesiology.
Processed book 1/700: The Church of Christ: A Biblical Ecclesiology for Today
summary generated for book: "Voices from The Farm" offers a compelling collection of first-hand accounts from the early d